# Student Enrollment

In [ ]:
import cv2
import os
import time
import csv

# Load Haar cascade for face detection
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def face_cropped(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return None  # No face detected

    for (x, y, w, h) in faces:
        return img[y:y+h, x:x+w]  # Return the first detected face

def save_student_info(user_id, name, phone, email):
    """Save student info to students.csv"""
    file_path = "students.csv"
    file_exists = os.path.isfile(file_path)

    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["ID", "Name", "Phone", "Email"])  # Write header
        writer.writerow([user_id, name, phone, email])

    print(f"✅ Student info saved: {user_id}, {name}, {phone}, {email}")

def generate_dataset(user_id=1, name="Unknown", phone="N/A", email="N/A", num_samples=100, delay=0.1):
    cap = cv2.VideoCapture(0)  # Open webcam
    if not cap.isOpened():
        print("Error: Could not access the camera.")
        exit()

    img_id = 0
    os.makedirs("data", exist_ok=True)  # Ensure "data" folder exists

    # Save student info
    save_student_info(user_id, name, phone, email)

    print("🔄 Capturing faces... Please stay still.")

    while img_id < num_samples:
        ret, frame = cap.read()
        if not ret:
            print("❌ Error: Failed to capture image.")
            break

        face = face_cropped(frame)
        if face is not None:
            img_id += 1
            face = cv2.resize(face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = os.path.join("data", f"user_{user_id}_{img_id}.jpg")
            cv2.imwrite(file_name_path, face)

            # Display capture progress
            cv2.putText(face, f"ID {user_id} - {img_id}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.imshow("Capturing Face", face)

            print(f"✅ Image {img_id}/{num_samples} captured...")

            time.sleep(delay)  # Slow down the capture process

        if cv2.waitKey(1) == 13 or img_id == num_samples:
            print("✅ Dataset collection completed successfully.")
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the function
if __name__ == "__main__":
    user_id = input("Enter user ID: ")
    name = input("Enter Name: ")
    phone = input("Enter Phone Number: ")
    email = input("Enter Email: ")
    generate_dataset(user_id=int(user_id), name=name, phone=phone, email=email, num_samples=100, delay=0.1)


✅ Student info saved: 564, Sorabh, 1234567, sorabh@gmail.com
🔄 Capturing faces... Please stay still.
✅ Image 1/100 captured...
✅ Image 2/100 captured...
✅ Image 3/100 captured...
✅ Image 4/100 captured...
✅ Image 5/100 captured...
✅ Image 6/100 captured...
✅ Image 7/100 captured...
✅ Image 8/100 captured...
✅ Image 9/100 captured...
✅ Image 10/100 captured...
✅ Image 11/100 captured...
✅ Image 12/100 captured...
✅ Image 13/100 captured...
✅ Image 14/100 captured...
✅ Image 15/100 captured...
✅ Image 16/100 captured...
✅ Image 17/100 captured...
✅ Image 18/100 captured...
✅ Image 19/100 captured...
✅ Image 20/100 captured...
✅ Image 21/100 captured...
✅ Image 22/100 captured...
✅ Image 23/100 captured...
✅ Image 24/100 captured...
✅ Image 25/100 captured...
✅ Image 26/100 captured...
✅ Image 27/100 captured...
✅ Image 28/100 captured...
✅ Image 29/100 captured...
✅ Image 30/100 captured...
✅ Image 31/100 captured...
✅ Image 32/100 captured...
✅ Image 33/100 captured...
✅ Image 34/100 ca

# Feature Extraction

In [30]:
import cv2
import numpy as np
import os

def train_model(data_folder="data", model_filename="trained_model.yml"):
    recognizer = cv2.face.LBPHFaceRecognizer_create()  # Create LBPH Face Recognizer
    faces = []
    ids = []

    # Get the list of images from the dataset
    image_files = [f for f in os.listdir(data_folder) if f.endswith(".jpg")]
    
    if len(image_files) == 0:
        print("❌ No images found in dataset. Please collect face samples first.")
        return

    print("🔄 Training model... Please wait.")

    for image_file in image_files:
        image_path = os.path.join(data_folder, image_file)

        # Read image in grayscale
        face_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Extract user ID from filename (e.g., "user_1_10.jpg" → ID = 1)
        user_id = int(image_file.split("_")[1])  # Extract ID from filename
        
        faces.append(face_img)
        ids.append(user_id)

    # Convert ID list to numpy array
    ids = np.array(ids)

    # Train the face recognizer
    recognizer.train(faces, ids)

    # Save the trained model
    recognizer.save(model_filename)

    print(f"✅ Model training completed! Model saved as '{model_filename}'.")

# Run the training function
if __name__ == "__main__":
    train_model()


🔄 Training model... Please wait.
✅ Model training completed! Model saved as 'trained_model.yml'.


# Attendance Marking

In [ ]:
import cv2
import numpy as np
import csv
import os

# Load Haar cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load trained model
recognizer = cv2.face.LBPHFaceRecognizer_create()
model_filename = "trained_model.yml"

if os.path.exists(model_filename):
    recognizer.read(model_filename)
    print(f"✅ Loaded trained model from '{model_filename}'.")
else:
    print(f"❌ Model file '{model_filename}' not found! Please train the model first.")
    exit()

# Load student details from CSV
student_data = {}
csv_file = "students.csv"
if os.path.exists(csv_file):
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            student_data[int(row[0])] = {"name": row[1], "phone": row[2], "email": row[3]}
else:
    print(f"❌ Student database '{csv_file}' not found!")

def recognize_face(frame):
    """Detect and recognize faces in a video frame."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        id, confidence = recognizer.predict(face_roi)

        confidence = int(100 * (1 - confidence / 300))

        if confidence > 75 and id in student_data:
            student = student_data[id]
            color = (0, 255, 0)  # Green for known users
        else:
            name = "UNKNOWN"
            color = (0, 0, 255)  # Red for unknown users

        # Draw rectangle and text
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame,  f"{name} ({confidence}%)", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    return frame

# Open webcam for real-time recognition
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Error: Could not access the camera.")
    exit()

print("📷 Face recognition started. Press 'Enter' to exit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Error: Failed to capture frame.")
        break

    frame = recognize_face(frame)
    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) == 13:  # Press 'Enter' to exit
        break

cap.release()
cv2.destroyAllWindows()
print("✅ Face recognition session ended.")


✅ Loaded trained model from 'trained_model.yml'.
📷 Face recognition started. Press 'Enter' to exit.
✅ Face recognition session ended.
